# Introduction to Spark

In [1]:
import findspark

In [2]:
findspark.init('/home/aforestier10/Downloads/spark-3.5.3-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName('Basics').getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

## The Basics

In [9]:
df = spark.read.json('people.json')

In [12]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [13]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [15]:
df.columns

['age', 'name']

In [18]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   NULL|
| stddev|7.7781745930520225|   NULL|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



### Define Schema

In [20]:
from pyspark.sql.types import (StructField, StringType, IntegerType, StructType)

In [25]:
# A list defining the types of each column in the dataframe
# Telling field "age" is of type Integer. True means that there can be nulls. False would enforce there being no nulls
data_schema = [StructField('age', IntegerType(), True),
              StructField('name', StringType(), True)]

In [26]:
# Define the final structure
final_struct = StructType(fields=data_schema)

In [27]:
# Now, read in data and use defined schema
df = spark.read.json('people.json', final_struct)

# Show its schema
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Get Data

In [28]:
# Grab column
df.select('age')

DataFrame[age: int]

In [29]:
# Show single column
df.select('age').show()

+----+
| age|
+----+
|NULL|
|  30|
|  19|
+----+



In [30]:
# First two head of dataframe - list of row objects
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [31]:
# Get list of columns
df.select(['age', 'name'])

DataFrame[age: int, name: string]

In [35]:
# Add column - returns new dataframe by adding or replacing existing column. Not in-place
df.withColumn('double_age', df['age'] * 2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|NULL|Michael|      NULL|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [39]:
# Rename column - not in place
df.withColumnRenamed('age', 'Age').show()

+----+-------+
| Age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Using pure SQL

In [40]:
# Register dataframe as temporary view
df.createOrReplaceTempView('people')

In [41]:
# now can query temporary view!
results = spark.sql("SELECT * FROM people")
results.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [45]:
query = "SELECT * FROM people WHERE name = 'Andy'"
new_results = spark.sql(query)
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

